In [1]:
%gui asyncio
import asyncio

In [12]:
from functools import wraps
def yield_for_change(widget):
    def f(iterator):
        @wraps(iterator)
        def inner():
            i = iterator()
            def next_i(change):
                try:
                    i.send(change)
                except StopIteration as e:
                    widget.unobserve(next_i, attribute)
            widget.on_click(next_i)
            # start the generator
            next(i)
        return inner
    return f

from ipywidgets import Button
button=Button()

def on_button_clicked():
    print("Button clicked.")

list_to_tag = ["one", "two", "three", "four", "five"]
@yield_for_change(button)
def f():
    for item in list_to_tag:
        print('tried to tag ', item)
        x = yield

f()
button

tried to tag  one


Button(style=ButtonStyle())

tried to tag  two
tried to tag  three
tried to tag  four
tried to tag  five


NameError: name 'attribute' is not defined

# try to modify the stack overflow example with a callback to another function.

In [11]:
# using yield_for_change for button
# this works for buttons, via https://stackoverflow.com/questions/55244865/pause-jupyter-notebook-widgets-waiting-for-user-input 

from functools import wraps
from ipywidgets import Button

def yield_for_change(widget):
    def f(iterator):
        @wraps(iterator)
        def inner():
            i = iterator()
            def next_i(change):
                try:
                    i.send(change.description)
                except StopIteration as e:
                    widget.unobserve(next_i, attribute)
            widget.on_click(next_i)
            # start the generator
            next(i)
        return inner
    return f

button=Button(description="wow")
button_other=Button(description="other")

def on_button_clicked():
    print("Button clicked.")
   
@yield_for_change(button_other)
def y():
    print("called another function")
    yval = yield 


list_to_tag = ["one", "two", "three", "four", "five"]
@yield_for_change(button)
def f():
    for item in list_to_tag:
        print('tried to tag ', item)
        x = yield
        print(x)
        if x == "wow":
            button_other
            y()

f()
button
# button works!

tried to tag  one


Button(description='wow', style=ButtonStyle())

wow
called another function
tried to tag  two
wow
called another function
tried to tag  three
wow
called another function
tried to tag  four


In [9]:
# yield_for_change for slider this works for a slider, from the widgets documentation 

from functools import wraps
from ipywidgets import IntSlider, VBox, HTML

def yield_for_change(widget, attribute):
    """Pause a generator to wait for a widget change event.

    This is a decorator for a generator function which pauses the generator on yield
    until the given widget attribute changes. The new value of the attribute is
    sent to the generator and is the value of the yield.
    """
    def f(iterator):
        @wraps(iterator)
        def inner():
            i = iterator()
            def next_i(change):
                try:
                    i.send(change.new)
                except StopIteration as e:
                    widget.unobserve(next_i, attribute)
            widget.observe(next_i, attribute)
            # start the generator
            next(i)
        return inner
    return f


slider2=IntSlider()

@yield_for_change(slider2, 'value')
def f():
    for i in range(10):
        print('did work %s'%i)
        x = yield
        print('generator function continued with value %s'%x)
f()

# slider2

did work 0


In [ ]:
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [10]:
# call function using await - slider - works
from ipywidgets import IntSlider
import asyncio

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

slider = IntSlider()

list_to_tag = ["one", "two", "three", "four", "five"]

async def f():
    """
    this function has a weird effect by which the first print is missed, and 
    nothing happens until we interact with the widget, can be gotten around. 
    """
    for i in list_to_tag:
        print('going to tag ', i)
        x = await wait_for_change(slider, 'value')
        print("tagged ", i, "with  %s"%x)
        print("")

asyncio.ensure_future(f())
slider
#slider

IntSlider(value=0)

tagged  one with  1

going to tag  two
tagged  two with  2

going to tag  three
tagged  three with  3

going to tag  four
tagged  four with  4

going to tag  five
tagged  five with  5



In [28]:
# call function using await - button - doesn't work
from ipywidgets import Button

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        print(change)
        future.set_result(change)
        widget.unobserve(getvalue, value)
    widget.on_click(getvalue)
    return future

button=Button(description="wow")

list_to_tag = ["one", "two", "three", "four"]

async def f():
    """
    this function has a weird effect by which the first print is missed, and 
    nothing happens until we interact with the widget, can be gotten around. 
    """
    for i in list_to_tag:
        print('going to tag ', i)
        x = await wait_for_change(button, 'value')
        print("tagged ", i, "with  %s"%x)
        print("")
asyncio.create_task(f())

# button

<Task pending coro=<f() running at <ipython-input-28-d35736310dc7>:18>>

In [35]:
# call function using await - button - 
from ipywidgets import Button

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        print(change)
        future.set_result(change)
        widget.unobserve(getvalue, value)
    widget.on_click(getvalue)
    return future

button=Button(description="wow")

list_to_tag = ["one", "two", "three", "four"]

async def f():
    """
    this function has a weird effect by which the first print is missed, and 
    nothing happens until we interact with the widget, can be gotten around. 
    """
    for i in list_to_tag:
        print('going to tag ', i)
        x = await wait_for_change(button, 'value')
        print("tagged ", i, "with  %s"%x)
        print("")
asyncio.create_task(f())

button

Button(description='wow', style=ButtonStyle())

Button(description='wow', style=ButtonStyle())
tagged  one with  Button(description='wow', style=ButtonStyle())

going to tag  two
Button(description='wow', style=ButtonStyle())


InvalidStateError: invalid state

Button(description='wow', style=ButtonStyle())
tagged  two with  Button(description='wow', style=ButtonStyle())

going to tag  three
